In [2]:
import json
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import sklearn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
with open('full_format_recipes.json', 'r') as f:
  data = json.load(f)
data[0]

{'directions': ['1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool.',
  '2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper.',
  '3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.'],
 'fat': 7.0,
 'date': '2006-09-01T04:00:00.000Z',
 'categories': ['Sandwich',
  'Bean',
  'Fruit',
  'Tomato',
  'turkey',

In [4]:
print(len(data))

20130


In [5]:
categories = []
for data_temp in data:
  if 'categories' in data_temp:
    for category in data_temp['categories']:
      if category not in categories:
        categories.append(category)

In [6]:
include_categories = ['Vegan']
data_vegan = []
for data_temp in data:
  if 'categories' in data_temp:
    for category in data_temp['categories']:
      if category in include_categories:
        data_vegan.append(data_temp)

In [7]:
print(len(data_vegan))

1851


In [8]:
categories_vegan = {}
for data_temp in data_vegan:
  if 'categories' in data_temp:
    for category in data_temp['categories']:
      if category not in categories_vegan:
        categories_vegan[category] = 1
      else:
        categories_vegan[category] += 1

In [9]:
categories_vegan = sorted(categories_vegan.items(), key=lambda x: x[1], reverse=True)

In [10]:
print(len(categories_vegan))
print(len(categories))
categories_vegan[:20]

422
674


[('Vegan', 1851),
 ('Vegetarian', 1730),
 ('Kosher', 1378),
 ('Pescatarian', 1368),
 ('Peanut Free', 1351),
 ('Dairy Free', 1326),
 ('Soy Free', 1286),
 ('Wheat/Gluten-Free', 1223),
 ('Tree Nut Free', 1196),
 ('Side', 941),
 ('Bon Appétit', 800),
 ('No Sugar Added', 730),
 ('Quick & Easy', 724),
 ('Gourmet', 637),
 ('Healthy', 588),
 ('Summer', 474),
 ('Sugar Conscious', 403),
 ('Kidney Friendly', 385),
 ('Paleo', 378),
 ('Salad', 333)]

In [11]:
df_all = pd.DataFrame(data, columns=data[0].keys())
df_all

,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium
0,"[1. Place the stock, lentils, celery, carrot, ...",7.0,2006-09-01T04:00:00.000Z,"[Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...",426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap","[4 cups low-sodium vegetable or chicken stock,...",559.0
1,[Combine first 9 ingredients in heavy medium s...,23.0,2004-08-20T04:00:00.000Z,"[Food Processor, Onion, Pork, Bake, Bastille D...",403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"[1 1/2 cups whipping cream, 2 medium onions, c...",1439.0
2,[In a large heavy saucepan cook diced fennel a...,7.0,2004-08-20T04:00:00.000Z,"[Soup/Stew, Dairy, Potato, Vegetable, Fennel, ...",165.0,None,6.0,3.750,Potato and Fennel Soup Hodge,"[1 fennel bulb (sometimes called anise), stalk...",165.0
3,[Heat oil in heavy large skillet over medium-h...,NaN,2009-03-27T04:00:00.000Z,"[Fish, Olive, Tomato, Sauté, Low Fat, Low Cal,...",NaN,The Sicilian-style tomato sauce has tons of Me...,NaN,5.000,Mahi-Mahi in Tomato Olive Sauce,"[2 tablespoons extra-virgin olive oil, 1 cup c...",NaN
4,[Preheat oven to 350°F. Lightly grease 8x8x2-i...,32.0,2004-08-20T04:00:00.000Z,"[Cheese, Dairy, Pasta, Vegetable, Side, Bake, ...",547.0,None,20.0,3.125,Spinach Noodle Casserole,"[1 12-ounce package frozen spinach soufflé, th...",452.0
...,...,...,...,...,...,...,...,...,...,...,...
20125,[Beat whites in a bowl with an electric mixer ...,2.0,2004-08-20T04:00:00.000Z,"[Mixer, Cheese, Egg, Fry, Cocktail Party, Parm...",28.0,None,2.0,3.125,Parmesan Puffs,"[2 large egg whites, 3 oz Parmigiano-Reggiano,...",64.0
20126,[Bring broth to simmer in saucepan.Remove from...,28.0,2008-02-28T22:06:54.000Z,"[Side, Kid-Friendly, High Fiber, Dinner, Parme...",671.0,Cooking the artichokes with the rice infuses t...,22.0,4.375,Artichoke and Parmesan Risotto,"[5 1/2 cups (or more) low-salt chicken broth, ...",583.0
20127,"[Using a sharp knife, cut a shallow X in botto...",38.0,2005-10-21T18:21:20.000Z,"[Onion, Poultry, turkey, Vegetable, Bake, Kid-...",563.0,None,31.0,4.375,Turkey Cream Puff Pie,"[1 small tomato, 1 small onion, finely chopped...",652.0
20128,[Heat 2 tablespoons oil in heavy medium skille...,24.0,2004-08-20T04:00:00.000Z,"[Milk/Cream, Citrus, Dairy, Fish, Garlic, Past...",631.0,"Sharon Hooykaas of Los Alamitos, California, w...",45.0,4.375,Snapper on Angel Hair with Citrus Cream,"[4 tablespoons olive oil, 4 shallots, thinly s...",517.0


In [12]:
df_vegan = pd.DataFrame(data_vegan, columns=data_vegan[0].keys())
df_vegan

,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium
0,"[Halve the potatoes, unpeeled, and cut them in...",19.0,2004-08-20T04:00:00.000Z,"[Salad, Blender, Olive, Onion, Potato, Side, R...",351.0,None,6.0,4.375,"Green Bean, Red Onion, and Roast Potato Salad ...","[3 pounds red boiling potatoes, 2/3 cup olive ...",79.0
1,[Cut off and discard 1 inch from stem ends of ...,10.0,2004-08-20T04:00:00.000Z,"[Garlic, Side, Sauté, Vegetarian, Quick & Easy...",107.0,Active time: 25 min Start to finish: 30 min,4.0,4.375,Sauteed Broccoli Rabe,"[2 lb broccoli rabe, 3 large garlic cloves, th...",329.0
2,[Cut peel and white pith from grapefruits and ...,1.0,2004-08-20T04:00:00.000Z,"[Salad, Breakfast, Brunch, No-Cook, Quick & Ea...",191.0,None,3.0,4.375,Citrus Salad with Mint Sugar,"[2 white grapefruits, 2 pink grapefruits, 6 la...",4.0
3,[Preheat oven to 475°F. Pierce eggplants with ...,7.0,2004-08-20T04:00:00.000Z,"[Condiment/Spread, Olive, Vegetable, Roast, Co...",177.0,None,5.0,3.750,Roasted Eggplant and Olive Spread with Pita Br...,"[3 1-pound eggplants, 3 6- to 7-inch-diameter ...",116.0
4,"[Grind the parsley, sage, rosemary, and thyme ...",12.0,2011-06-23T04:00:00.000Z,"[Microwave, Bake, Vegetarian, Wheat/Gluten-Fre...",224.0,Wheat Free,21.0,3.125,Scarborough Fair Tofu Burger,"[1 tablespoon (2 g) dried parsley, 1 tablespoo...",340.0
...,...,...,...,...,...,...,...,...,...,...,...
1846,[In a saucepan bring vinegar and water to a bo...,11.0,2004-08-20T04:00:00.000Z,"[Salad, Pepper, Vegetable, Side, Sauté, Vegeta...",193.0,Can be prepared in 45 minutes or less.,2.0,3.750,Wilted Red Cabbage and Bell Pepper Slaw,"[1/2 cup distilled white vinegar, 1/2 cup wate...",461.0
1847,"[Preheat the oven to 375°F., Have an ice bath ...",27.0,2012-08-24T04:00:00.000Z,"[Salad, Side, Vegetarian, Lunch, Avocado, Carr...",293.0,I love using baby carrots in assorted colors. ...,4.0,3.125,Roasted and Raw Carrot Salad with Avocado and ...,[2 bunches (about 10 ounces total) baby carrot...,647.0
1848,[Makes enough spice rub for two whole chickens...,0.0,2013-06-10T04:00:00.000Z,"[Low Fat, Vegetarian, Quick & Easy, Spice, Veg...",16.0,You can use this simple formula to wake up the...,0.0,4.375,4-3-2-1 Spice Rub,"[4 tablespoons kosher salt, 3 tablespoons brow...",2292.0
1849,"[Bring all ingredients to a simmer, covered di...",NaN,2006-05-16T20:12:06.000Z,"[Food Processor, Fruit, Side, Vegetarian, Date...",NaN,This recipe is an accompaniment for Rack of La...,NaN,2.500,Date Purée,"[3 cups pitted dates (14 ounces), 1 (750-ml) b...",NaN


In [13]:
df_all = df_all.dropna(subset=['categories', 'title'])
df_vegan = df_vegan.dropna(subset=['categories', 'title'])

df_all = df_all.drop('date', axis=1)
df_vegan = df_vegan.drop('date', axis=1)

In [14]:
df_all['is_vegan'] = df_all['categories'].apply(lambda x: 1 if 'Vegan' in x else 0)
df_all

,directions,fat,categories,calories,desc,protein,rating,title,ingredients,sodium,is_vegan
0,"[1. Place the stock, lentils, celery, carrot, ...",7.0,"[Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...",426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap","[4 cups low-sodium vegetable or chicken stock,...",559.0,0
1,[Combine first 9 ingredients in heavy medium s...,23.0,"[Food Processor, Onion, Pork, Bake, Bastille D...",403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"[1 1/2 cups whipping cream, 2 medium onions, c...",1439.0,0
2,[In a large heavy saucepan cook diced fennel a...,7.0,"[Soup/Stew, Dairy, Potato, Vegetable, Fennel, ...",165.0,None,6.0,3.750,Potato and Fennel Soup Hodge,"[1 fennel bulb (sometimes called anise), stalk...",165.0,0
3,[Heat oil in heavy large skillet over medium-h...,NaN,"[Fish, Olive, Tomato, Sauté, Low Fat, Low Cal,...",NaN,The Sicilian-style tomato sauce has tons of Me...,NaN,5.000,Mahi-Mahi in Tomato Olive Sauce,"[2 tablespoons extra-virgin olive oil, 1 cup c...",NaN,0
4,[Preheat oven to 350°F. Lightly grease 8x8x2-i...,32.0,"[Cheese, Dairy, Pasta, Vegetable, Side, Bake, ...",547.0,None,20.0,3.125,Spinach Noodle Casserole,"[1 12-ounce package frozen spinach soufflé, th...",452.0,0
...,...,...,...,...,...,...,...,...,...,...,...
20125,[Beat whites in a bowl with an electric mixer ...,2.0,"[Mixer, Cheese, Egg, Fry, Cocktail Party, Parm...",28.0,None,2.0,3.125,Parmesan Puffs,"[2 large egg whites, 3 oz Parmigiano-Reggiano,...",64.0,0
20126,[Bring broth to simmer in saucepan.Remove from...,28.0,"[Side, Kid-Friendly, High Fiber, Dinner, Parme...",671.0,Cooking the artichokes with the rice infuses t...,22.0,4.375,Artichoke and Parmesan Risotto,"[5 1/2 cups (or more) low-salt chicken broth, ...",583.0,0
20127,"[Using a sharp knife, cut a shallow X in botto...",38.0,"[Onion, Poultry, turkey, Vegetable, Bake, Kid-...",563.0,None,31.0,4.375,Turkey Cream Puff Pie,"[1 small tomato, 1 small onion, finely chopped...",652.0,0
20128,[Heat 2 tablespoons oil in heavy medium skille...,24.0,"[Milk/Cream, Citrus, Dairy, Fish, Garlic, Past...",631.0,"Sharon Hooykaas of Los Alamitos, California, w...",45.0,4.375,Snapper on Angel Hair with Citrus Cream,"[4 tablespoons olive oil, 4 shallots, thinly s...",517.0,0


In [15]:
df_all['is_vegan'].value_counts()

0    18260
1     1851
Name: is_vegan, dtype: int64

In [16]:
len(df_vegan)

1851

In [17]:
convert_dict = {'directions': str,
                'fat': float,
                'categories': str,
                'ingredients': str,
                'calories': float,
                'protein': float,
                'rating': float,
                'sodium': float,
            }

In [18]:
df_all = df_all.astype(convert_dict)
df_vegan = df_vegan.astype(convert_dict)

df_all = df_all.drop_duplicates()
df_vegan = df_vegan.drop_duplicates()
df_vegan = df_vegan.dropna()

In [19]:
df_vegan = df_vegan.reset_index()
df_vegan = df_vegan.drop(columns='index')
df_vegan

,directions,fat,categories,calories,desc,protein,rating,title,ingredients,sodium
0,['Cut off and discard 1 inch from stem ends of...,10.0,"['Garlic', 'Side', 'Sauté', 'Vegetarian', 'Qui...",107.0,Active time: 25 min Start to finish: 30 min,4.0,4.375,Sauteed Broccoli Rabe,"['2 lb broccoli rabe', '3 large garlic cloves,...",329.0
1,"['Grind the parsley, sage, rosemary, and thyme...",12.0,"['Microwave', 'Bake', 'Vegetarian', 'Wheat/Glu...",224.0,Wheat Free,21.0,3.125,Scarborough Fair Tofu Burger,"['1 tablespoon (2 g) dried parsley', '1 tables...",340.0
2,['Trim root end of each onion and cut an X in ...,0.0,"['Onion', 'Breakfast', 'Side', 'Thanksgiving',...",171.0,"In this chutney-like compote, quince lends a r...",1.0,3.750,"Cranberry, Quince, and Pearl Onion Compote",['1/2 pound pearl onions (preferably red; 2 cu...,6.0
3,['In a heavy saucepan cook the onion and the g...,7.0,"['Sauce', 'Garlic', 'Onion', 'Tomato', 'Vegeta...",112.0,This recipe is used to prepare Cod Cannelloni ...,2.0,0.000,Spicy Tomato Sauce,"['1 small onion, chopped fine', '2 large garli...",12.0
4,['Using a long-handled fork char the peppers o...,7.0,"['Side', 'Broil', 'Vegetarian', 'Quick & Easy'...",123.0,This recipe is used to prepare Cod Cannelloni ...,5.0,3.750,Swiss Chard with Roasted Pepper,"['2 1/2 pound Swiss chard, washed well, the co...",605.0
...,...,...,...,...,...,...,...,...,...,...
902,['Combine celery and spinach in large bowl. Dr...,9.0,"['Salad', 'Appetizer', 'Side', 'No-Cook', 'Veg...",91.0,Asiago frico — easy-to-make crispy cheese wafe...,1.0,4.375,Spinach and Celery Salad with Lemon Vinaigrette,"['4 large celery stalks, very thinly sliced on...",35.0
903,['Combine spring onion and vinegar in a small ...,10.0,"['Salad', 'Onion', 'Tomato', 'Side', 'Vegetari...",153.0,"For this juicy, herbaceous salad, feel free to...",3.0,4.375,Shirazi Salad,"['1 medium spring onion, thinly sliced', '1/4 ...",23.0
904,['Prepare barbecue (medium-high heat). Place e...,5.0,"['Garlic', 'Vegetable', 'Side', 'Vegetarian', ...",94.0,"The all-purpose side: It goes with the lamb, t...",2.0,3.750,Grilled Eggplant and Bell Peppers with Roasted...,"['1 large eggplant (1 1/4 to 1 1/2 pounds), un...",11.0
905,['In a saucepan bring vinegar and water to a b...,11.0,"['Salad', 'Pepper', 'Vegetable', 'Side', 'Saut...",193.0,Can be prepared in 45 minutes or less.,2.0,3.750,Wilted Red Cabbage and Bell Pepper Slaw,"['1/2 cup distilled white vinegar', '1/2 cup w...",461.0


In [20]:
print(f'length all data = {len(df_all)}\n')
print(f'null all data = \n{df_all.isnull().sum()}\n')
print(f'length vegan data = {len(df_vegan)}\n')
print(f'null vegan data = \n{df_vegan.isnull().sum()}\n')

length all data = 18295

null all data = 
directions        0
fat            3816
categories        0
calories       3763
desc           6081
protein        3796
rating           10
title             0
ingredients       0
sodium         3764
is_vegan          0
dtype: int64

length vegan data = 907

null vegan data = 
directions     0
fat            0
categories     0
calories       0
desc           0
protein        0
rating         0
title          0
ingredients    0
sodium         0
dtype: int64



### Preprocess Title with Stopwords

In [21]:
df_vegan['title'].iloc[0]

'Sauteed Broccoli Rabe '

In [22]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hufah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
stopwords_uses = stopwords.words("english")

In [24]:
def preprocess_title(sentence):
    # end space
    sentence = re.sub(r'\s$', '', sentence)

    # Special Characters
    sentence = re.sub(r'[^a-zA-Z0-9 ]', '', sentence).lower()

    # Stopwords
    sentence = ' '.join([word for word in sentence.split() if word not in stopwords_uses])
    return sentence

In [25]:
preprocess_title(df_all['title'].iloc[0])

'lentil apple turkey wrap'

## Modelling

In [26]:
input_title = df_all['title'].apply(lambda x: preprocess_title(x))
targets = df_all['is_vegan']

In [27]:
input_title

0                     lentil apple turkey wrap
1        boudin blanc terrine red onion confit
2                     potato fennel soup hodge
3                  mahimahi tomato olive sauce
4                     spinach noodle casserole
                         ...                  
20125                           parmesan puffs
20126               artichoke parmesan risotto
20127                    turkey cream puff pie
20128          snapper angel hair citrus cream
20129     baked ham marmaladehorseradish glaze
Name: title, Length: 18295, dtype: object

In [28]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(input_title)
input_seq = tokenizer.texts_to_sequences(input_title)

word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'salad': 2, 'sauce': 3, 'chicken': 4, 'cheese': 5, 'cream': 6, 'grilled': 7, 'soup': 8, 'roasted': 9, 'red': 10, 'tomato': 11, 'lemon': 12, 'chocolate': 13, 'pork': 14, 'potato': 15, 'cake': 16, 'shrimp': 17, 'green': 18, 'garlic': 19, 'lamb': 20, 'pepper': 21, 'butter': 22, 'rice': 23, 'orange': 24, 'potatoes': 25, 'beef': 26, 'turkey': 27, 'corn': 28, 'roast': 29, 'dressing': 30, 'pie': 31, 'apple': 32, 'ginger': 33, 'onion': 34, 'salmon': 35, 'bean': 36, 'bacon': 37, 'tomatoes': 38, 'vinaigrette': 39, 'sweet': 40, 'spicy': 41, 'spinach': 42, 'vegetable': 43, 'salsa': 44, 'bread': 45, 'goat': 46, 'fennel': 47, 'spiced': 48, 'pasta': 49, 'white': 50, 'onions': 51, 'fresh': 52, 'tart': 53, 'ice': 54, 'mustard': 55, 'squash': 56, 'baked': 57, 'mushroom': 58, 'mint': 59, 'sausage': 60, 'beans': 61, 'coconut': 62, 'mushrooms': 63, 'lime': 64, 'arugula': 65, 'smoked': 66, 'steak': 67, 'vegetables': 68, 'yogurt': 69, 'chops': 70, 'fruit': 71, 'bell': 72, 'rosemary': 73, 'fried'

In [29]:
maxlen = max([len(word) for word in input_seq])
pad_type = 'post'
trunc_type = 'post'
print(maxlen)

14


In [30]:
input_padded = pad_sequences(input_seq, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
input_padded

array([[ 301,   32,   27, ...,    0,    0,    0],
       [2115, 1002,  516, ...,    0,    0,    0],
       [  15,   47,    8, ...,    0,    0,    0],
       ...,
       [  27,    6,  721, ...,    0,    0,    0],
       [ 336,  662, 1100, ...,    0,    0,    0],
       [  57,   89, 6824, ...,    0,    0,    0]])

In [31]:
embedding_dim = 100
filters = 32
kernel_size = 5
vocab_size = len(word_index) + 1
print(vocab_size)

6825


In [32]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(None,)))
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
model.add(tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         682500    
                                                                 
 conv1d (Conv1D)             (None, None, 32)          16032     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 32)         0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 128)              49664     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0

In [33]:
embd = model.layers[0]
conv = model.layers[1]
pool = model.layers[2]
bid = model.layers[3]

print(embd)
print(conv)
print(pool)
print(bid)

In [34]:
def preprocess_texts_input(sentence):
    sentence = [preprocess_title(sentence)]
    sentence = tokenizer.texts_to_sequences(sentence)
    sentence = pad_sequences(sentence, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
    return sentence

In [35]:
text_input_0 = df_vegan['title'].iloc[0]
print(text_input_0)
text_input_0 = preprocess_texts_input(text_input_0)
text_input_0

Sauteed Broccoli Rabe 


array([[125, 121, 322,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0]])

In [36]:
text_input = tf.keras.layers.Input(shape=(None,))
x = tf.expand_dims(text_input, axis=-1)
x = embd(text_input)
x = conv(x)
x = pool(x)
x = bid(x)
embedding_model = tf.keras.Model(text_input, x)
embedding_model.compile()
embedding_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         682500    
                                                                 
 conv1d (Conv1D)             (None, None, 32)          16032     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 32)         0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 128)              49664     
 l)                                                              
                                                                 
Total params: 748,196
Trainable params: 748,196
Non-trainable

In [37]:
epochs = 100

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5, min_delta=0.003)
history = model.fit(x=input_padded, y=targets, epochs=epochs, verbose=1, callbacks=earlyStopping)

Epoch 1/100
572/572 [==============================] - 8s 8ms/step - loss: 0.2609 - accuracy: 0.9125
Epoch 2/100
572/572 [==============================] - 4s 8ms/step - loss: 0.1810 - accuracy: 0.9163
Epoch 3/100
572/572 [==============================] - 4s 8ms/step - loss: 0.1483 - accuracy: 0.9305
Epoch 4/100
572/572 [==============================] - 4s 8ms/step - loss: 0.1197 - accuracy: 0.9430
Epoch 5/100
572/572 [==============================] - 4s 8ms/step - loss: 0.0977 - accuracy: 0.9553
Epoch 6/100
572/572 [==============================] - 5s 8ms/step - loss: 0.0773 - accuracy: 0.9665
Epoch 7/100
572/572 [==============================] - 5s 8ms/step - loss: 0.0609 - accuracy: 0.9729
Epoch 8/100
572/572 [==============================] - 6s 10ms/step - loss: 0.0481 - accuracy: 0.9794
Epoch 9/100
572/572 [==============================] - 8s 13ms/step - loss: 0.0424 - accuracy: 0.9825
Epoch 10/100
572/572 [==============================] - 6s 10ms/step - loss: 0.0337 - acc

In [38]:
def find_encoding(sentence):
    sentence = preprocess_texts_input(sentence)
    encoding = embedding_model(sentence)
    return encoding

In [39]:
df_vegan_test = df_vegan.copy()
df_vegan_test

,directions,fat,categories,calories,desc,protein,rating,title,ingredients,sodium
0,['Cut off and discard 1 inch from stem ends of...,10.0,"['Garlic', 'Side', 'Sauté', 'Vegetarian', 'Qui...",107.0,Active time: 25 min Start to finish: 30 min,4.0,4.375,Sauteed Broccoli Rabe,"['2 lb broccoli rabe', '3 large garlic cloves,...",329.0
1,"['Grind the parsley, sage, rosemary, and thyme...",12.0,"['Microwave', 'Bake', 'Vegetarian', 'Wheat/Glu...",224.0,Wheat Free,21.0,3.125,Scarborough Fair Tofu Burger,"['1 tablespoon (2 g) dried parsley', '1 tables...",340.0
2,['Trim root end of each onion and cut an X in ...,0.0,"['Onion', 'Breakfast', 'Side', 'Thanksgiving',...",171.0,"In this chutney-like compote, quince lends a r...",1.0,3.750,"Cranberry, Quince, and Pearl Onion Compote",['1/2 pound pearl onions (preferably red; 2 cu...,6.0
3,['In a heavy saucepan cook the onion and the g...,7.0,"['Sauce', 'Garlic', 'Onion', 'Tomato', 'Vegeta...",112.0,This recipe is used to prepare Cod Cannelloni ...,2.0,0.000,Spicy Tomato Sauce,"['1 small onion, chopped fine', '2 large garli...",12.0
4,['Using a long-handled fork char the peppers o...,7.0,"['Side', 'Broil', 'Vegetarian', 'Quick & Easy'...",123.0,This recipe is used to prepare Cod Cannelloni ...,5.0,3.750,Swiss Chard with Roasted Pepper,"['2 1/2 pound Swiss chard, washed well, the co...",605.0
...,...,...,...,...,...,...,...,...,...,...
902,['Combine celery and spinach in large bowl. Dr...,9.0,"['Salad', 'Appetizer', 'Side', 'No-Cook', 'Veg...",91.0,Asiago frico — easy-to-make crispy cheese wafe...,1.0,4.375,Spinach and Celery Salad with Lemon Vinaigrette,"['4 large celery stalks, very thinly sliced on...",35.0
903,['Combine spring onion and vinegar in a small ...,10.0,"['Salad', 'Onion', 'Tomato', 'Side', 'Vegetari...",153.0,"For this juicy, herbaceous salad, feel free to...",3.0,4.375,Shirazi Salad,"['1 medium spring onion, thinly sliced', '1/4 ...",23.0
904,['Prepare barbecue (medium-high heat). Place e...,5.0,"['Garlic', 'Vegetable', 'Side', 'Vegetarian', ...",94.0,"The all-purpose side: It goes with the lamb, t...",2.0,3.750,Grilled Eggplant and Bell Peppers with Roasted...,"['1 large eggplant (1 1/4 to 1 1/2 pounds), un...",11.0
905,['In a saucepan bring vinegar and water to a b...,11.0,"['Salad', 'Pepper', 'Vegetable', 'Side', 'Saut...",193.0,Can be prepared in 45 minutes or less.,2.0,3.750,Wilted Red Cabbage and Bell Pepper Slaw,"['1/2 cup distilled white vinegar', '1/2 cup w...",461.0


In [40]:
df_vegan_test['encoding'] = df_vegan_test['title'].apply(lambda x: find_encoding(x))
df_vegan_test

,directions,fat,categories,calories,desc,protein,rating,title,ingredients,sodium,encoding
0,['Cut off and discard 1 inch from stem ends of...,10.0,"['Garlic', 'Side', 'Sauté', 'Vegetarian', 'Qui...",107.0,Active time: 25 min Start to finish: 30 min,4.0,4.375,Sauteed Broccoli Rabe,"['2 lb broccoli rabe', '3 large garlic cloves,...",329.0,"((tf.Tensor(-0.004997561, shape=(), dtype=floa..."
1,"['Grind the parsley, sage, rosemary, and thyme...",12.0,"['Microwave', 'Bake', 'Vegetarian', 'Wheat/Glu...",224.0,Wheat Free,21.0,3.125,Scarborough Fair Tofu Burger,"['1 tablespoon (2 g) dried parsley', '1 tables...",340.0,"((tf.Tensor(-0.032873765, shape=(), dtype=floa..."
2,['Trim root end of each onion and cut an X in ...,0.0,"['Onion', 'Breakfast', 'Side', 'Thanksgiving',...",171.0,"In this chutney-like compote, quince lends a r...",1.0,3.750,"Cranberry, Quince, and Pearl Onion Compote",['1/2 pound pearl onions (preferably red; 2 cu...,6.0,"((tf.Tensor(-0.033082686, shape=(), dtype=floa..."
3,['In a heavy saucepan cook the onion and the g...,7.0,"['Sauce', 'Garlic', 'Onion', 'Tomato', 'Vegeta...",112.0,This recipe is used to prepare Cod Cannelloni ...,2.0,0.000,Spicy Tomato Sauce,"['1 small onion, chopped fine', '2 large garli...",12.0,"((tf.Tensor(-0.0042622285, shape=(), dtype=flo..."
4,['Using a long-handled fork char the peppers o...,7.0,"['Side', 'Broil', 'Vegetarian', 'Quick & Easy'...",123.0,This recipe is used to prepare Cod Cannelloni ...,5.0,3.750,Swiss Chard with Roasted Pepper,"['2 1/2 pound Swiss chard, washed well, the co...",605.0,"((tf.Tensor(-0.010963243, shape=(), dtype=floa..."
...,...,...,...,...,...,...,...,...,...,...,...
902,['Combine celery and spinach in large bowl. Dr...,9.0,"['Salad', 'Appetizer', 'Side', 'No-Cook', 'Veg...",91.0,Asiago frico — easy-to-make crispy cheese wafe...,1.0,4.375,Spinach and Celery Salad with Lemon Vinaigrette,"['4 large celery stalks, very thinly sliced on...",35.0,"((tf.Tensor(-0.008308005, shape=(), dtype=floa..."
903,['Combine spring onion and vinegar in a small ...,10.0,"['Salad', 'Onion', 'Tomato', 'Side', 'Vegetari...",153.0,"For this juicy, herbaceous salad, feel free to...",3.0,4.375,Shirazi Salad,"['1 medium spring onion, thinly sliced', '1/4 ...",23.0,"((tf.Tensor(-0.010910619, shape=(), dtype=floa..."
904,['Prepare barbecue (medium-high heat). Place e...,5.0,"['Garlic', 'Vegetable', 'Side', 'Vegetarian', ...",94.0,"The all-purpose side: It goes with the lamb, t...",2.0,3.750,Grilled Eggplant and Bell Peppers with Roasted...,"['1 large eggplant (1 1/4 to 1 1/2 pounds), un...",11.0,"((tf.Tensor(-0.04781153, shape=(), dtype=float..."
905,['In a saucepan bring vinegar and water to a b...,11.0,"['Salad', 'Pepper', 'Vegetable', 'Side', 'Saut...",193.0,Can be prepared in 45 minutes or less.,2.0,3.750,Wilted Red Cabbage and Bell Pepper Slaw,"['1/2 cup distilled white vinegar', '1/2 cup w...",461.0,"((tf.Tensor(-0.019667884, shape=(), dtype=floa..."


In [41]:
df_vegan_test['encoding'].iloc[0]

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-0.00499756, -0.01832577,  0.01402648,  0.00069323, -0.01582474,
         0.0157007 ,  0.02070605, -0.02081669, -0.03601096,  0.00544928,
        -0.00248826, -0.00838793, -0.01624237,  0.01450848, -0.00669768,
        -0.01064341,  0.03635087,  0.02764704, -0.00767182, -0.00507099,
        -0.00065506,  0.00246916,  0.01990437, -0.02471892, -0.00162314,
         0.00191738,  0.02197388,  0.00069768, -0.00454371,  0.01477016,
         0.00050981,  0.01016123, -0.00297212, -0.00378206, -0.00709332,
         0.01685775,  0.00389121, -0.03118232,  0.01128873, -0.01735636,
        -0.00663428, -0.00155637, -0.00311429,  0.01220415,  0.02980531,
         0.00497331,  0.02029681, -0.00244489,  0.02043449,  0.02394096,
        -0.00731903, -0.00031133, -0.00583812, -0.01668152, -0.00356684,
         0.00169577,  0.00472964,  0.01499102,  0.01756978,  0.02095474,
         0.00418244, -0.00802123,  0.02852796,  0.00168769,  0.00116561,
 

In [42]:
test_df_vegan = df_vegan_test.loc[df_vegan_test['title'] == 'Spinach with Pine Nuts and Raisins ']
query = test_df_vegan['title'].iloc[0]
query

'Spinach with Pine Nuts and Raisins '

In [43]:
query = find_encoding(query)
query

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-1.29606929e-02, -3.79605629e-02,  3.62150557e-03,
         4.72939666e-03, -6.54325485e-02,  2.91095804e-02,
         1.39222266e-02, -1.80368423e-02, -7.46455863e-02,
         7.09770340e-03,  1.79976586e-03, -1.04313381e-01,
         3.67980823e-02,  3.94149311e-02, -9.49981622e-03,
         1.64201309e-03,  6.78195134e-02,  5.56495823e-02,
         2.93925707e-03,  3.30744311e-02, -1.77661004e-03,
        -7.52961775e-03,  1.11307651e-02, -3.44758444e-02,
         4.11914196e-03, -4.66644531e-04,  2.56222263e-02,
        -2.68673617e-02, -1.86380148e-02,  2.94126594e-03,
         2.31017475e-03,  1.44608887e-02,  1.90026946e-02,
         3.53469215e-02, -3.51227596e-02,  1.90038588e-02,
        -4.84677181e-02, -3.67252082e-02,  1.52928606e-02,
        -1.42566534e-02,  8.25516880e-03, -3.12467013e-02,
        -2.74510961e-02,  1.21569606e-02,  8.13282952e-02,
         3.54757681e-02, -3.18599306e-03,  1.45430835e-02,
      

In [44]:
df_vegan_test['similarity_score'] = df_vegan_test['encoding'].apply(lambda x: sklearn.metrics.pairwise.cosine_similarity(x, query)[0][0])
df_vegan_test = df_vegan_test.sort_values(by=['similarity_score'], ascending=False)
df_vegan_test[:20]

,directions,fat,categories,calories,desc,protein,rating,title,ingredients,sodium,encoding,similarity_score
25,['Rinse the spinach well and remove the stems....,10.0,"['Side', 'Sauté', 'Vegetarian', 'Rosh Hashanah...",159.0,(Spinaci con Pinoli e Passerine),7.0,4.375,Spinach with Pine Nuts and Raisins,"['2 1/2 pounds spinach', '2 to 3 tablespoons o...",539.0,"((tf.Tensor(-0.012960693, shape=(), dtype=floa...",1.000000
13,"['Pur&eaacute;e tomatillos, half of tomatoes, ...",5.0,"['Onion', 'Tomato', 'Appetizer', 'Dinner', 'Lu...",80.0,Celebrate the last of the hot weather with a s...,2.0,4.375,Tomato and Tomatillo Gazpacho,"['1/2 pound fresh tomatillos, husked, rinsed, ...",10.0,"((tf.Tensor(-0.0076212515, shape=(), dtype=flo...",0.980111
196,['Cook sugar in a dry 1-quart heavy saucepan o...,0.0,"['Sauce', 'Fruit Juice', 'Rum', 'Citrus', 'Des...",384.0,This recipe is an accompaniment for Toasted Co...,0.0,2.500,Orange Rum Caramel Sauce,"['1 cup sugar', '1/2 cup fresh orange juice', ...",2.0,"((tf.Tensor(-0.019143475, shape=(), dtype=floa...",0.966875
586,"['Cut peel, including all white pith, from 2 g...",2.0,"['Citrus', 'Fruit', 'Dessert', 'Vegetarian', '...",610.0,"For something refreshing after a rich meal, lo...",10.0,3.125,Grapefruit with Campari Syrup,['3 large pink or ruby red grapefruit (4 pound...,0.0,"((tf.Tensor(-0.011635465, shape=(), dtype=floa...",0.956563
507,['In a 1 1/2-quart heavy saucepan cook pine nu...,16.0,"['Rice', 'Side', 'Kid-Friendly', 'Quick & Easy...",272.0,This recipe can be prepared in 45 minutes or l...,5.0,4.375,Pine Nut and Basil Rice,"['1/4 cup pine nuts', '2 teaspoons olive oil',...",4.0,"((tf.Tensor(-0.013973975, shape=(), dtype=floa...",0.944587
37,['Combine first 7 ingredients in large saucepa...,9.0,"['Soup/Stew', 'Garlic', 'Ginger', 'Leafy Green...",152.0,Can be prepared in 45 minutes or less.,12.0,4.375,Chinese-Style Spinach and Mushroom Soup,"['2 14 1/2-ounce cans vegetable broth', '1 1/2...",1305.0,"((tf.Tensor(-0.022361113, shape=(), dtype=floa...",0.936735
459,"['In a bowl whisk together lemon juice, oil, a...",16.0,"['Salad', 'Bean', 'Leafy Green', 'Tomato', 'Si...",486.0,Can be prepared in 45 minutes or less.,23.0,4.375,"Black Bean, Corn, and Tomato Salad","['3 tablespoons fresh lemon juice', '2 tablesp...",327.0,"((tf.Tensor(-0.009317241, shape=(), dtype=floa...",0.934889
319,['Cook garlic in oil in a 2- to 3-quart wide h...,12.0,"['Citrus', 'Olive', 'Side', 'Vegetarian', 'Qui...",372.0,Couscous is one of the fastest side dishes to ...,9.0,3.750,Couscous with Olives and Lemon,"['1 1/2 teaspoons minced garlic', '3 tablespoo...",239.0,"((tf.Tensor(-0.013536495, shape=(), dtype=floa...",0.925386
630,"['Combine walnuts, 1 tablespoon water, sugar, ...",8.0,"['Salad', 'Fruit', 'Leafy Green', 'Nut', 'Side...",87.0,This recipe is an accompaniment for Cabrales C...,1.0,3.750,Endive and Asian Pear Salad,"['1/2 cup coarsely chopped walnuts', '1 tables...",15061.0,"((tf.Tensor(-0.019772483, shape=(), dtype=floa...",0.925119
714,['Rinse rice in a fine-mesh sieve until water ...,0.0,"['Rice', 'Side', 'Vegetarian', 'Dinner', 'Lunc...",263.0,Long-grain rice won't do the trick here; short...,5.0,5.000,Steamed Asian White Rice,['3 cups short- or medium-grain Asian white ri...,5.0,"((tf.Tensor(-0.01719251, shape=(), dtype=float...",0.924089


In [45]:
embedding_model.save('Trained E100, BiLSTM1, MaxP F32 KS5.h5')